In [1]:
!pip install together

In [ ]:
import pandas as pd
from together import Together
import os

client = Together(api_key="")  # Replace with your Together API key

def translate_text(client, text, source_language):
    """
    Translates a given text to English from the specified source language using Together API.

    Args:
        client: Together API client instance.
        text (str): The text to be translated.
        source_language (str): The source language (e.g., 'Hindi', 'Spanish', 'French').

    Returns:
        str: The translated text.
    """
    prompt = f"Translate the following sentence from {source_language} to English: {text}. Make sure the translation is accurate and simple."
    try:
        response = client.chat.completions.create(
            model="mistralai/Mixtral-8x7B-Instruct-v0.1",  # Model from Together API
            messages=[{"role": "user", "content": prompt}],
            stream=False
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"Error translating text from {source_language} to English: {e}")
        return f"Error: {e}"

def process_csv_with_translations(input_csv, output_csv, hindi_column, spanish_column, french_column):
    """
    Processes a CSV file with Hindi, Spanish, and French columns, and translates the text into English for each.
    Ensures that if the process is interrupted, the output file still gets saved and appends to existing file if present.

    Args:
        input_csv (str): Path to the input CSV file.
        output_csv (str): Path to save the output CSV file.
        hindi_column (str): Name of the column containing Hindi text.
        spanish_column (str): Name of the column containing Spanish text.
        french_column (str): Name of the column containing French text.
    """
    # Load the input CSV
    df = pd.read_csv(input_csv)

    if hindi_column not in df.columns or spanish_column not in df.columns or french_column not in df.columns:
        raise ValueError(f"One or more specified columns do not exist in the CSV.")

    # Check if output file exists and load it, otherwise start fresh
    if os.path.exists(output_csv):
        existing_df = pd.read_csv(output_csv)
        # Ensure we're appending by matching column names
        if 'Hindi_to_English' in existing_df.columns and 'Spanish_to_English' in existing_df.columns and 'French_to_English' in existing_df.columns:
            df = pd.concat([existing_df, df], ignore_index=True)
        else:
            # Add missing columns if the output file exists but doesn't have them
            df['Hindi_to_English'] = None
            df['Spanish_to_English'] = None
            df['French_to_English'] = None
    else:
        # Initialize new columns for translations if the file doesn't exist
        df['Hindi_to_English'] = None
        df['Spanish_to_English'] = None
        df['French_to_English'] = None

    # Process each row
    for idx, row in df.iterrows():
        try:
            print(f"Processing row {idx + 1}/{len(df)}")

            # Translate the text in Hindi, Spanish, and French to English
            df.at[idx, 'Hindi_to_English'] = translate_text(client, row[hindi_column], 'Hindi')
            df.at[idx, 'Spanish_to_English'] = translate_text(client, row[spanish_column], 'Spanish')
            df.at[idx, 'French_to_English'] = translate_text(client, row[french_column], 'French')

        except Exception as e:
            print(f"Error processing row {idx + 1}: {e}")
            df.at[idx, 'Hindi_to_English'] = f"Error: {e}"
            df.at[idx, 'Spanish_to_English'] = f"Error: {e}"
            df.at[idx, 'French_to_English'] = f"Error: {e}"

        # Save intermediate results after each translation to ensure data is not lost
        try:
            df.to_csv(output_csv, index=False)
            print(f"Progress saved to {output_csv}")
        except Exception as save_error:
            print(f"Error saving file: {save_error}")

    # Final save after all translations are done
    try:
        df.to_csv(output_csv, index=False)
        print(f"Translations saved to {output_csv}")
    except Exception as e:
        print(f"Error saving final file: {e}")

# Example usage
if __name__ == "__main__":
    input_csv_path = "/content/drive/MyDrive/GT/CS 8803 LLM/Hyperbaton Proj/hyperTrans3Inputs - Sheet1.csv"  # Replace with the path to your input CSV
    output_csv_path = "/content/drive/MyDrive/GT/CS 8803 LLM/Hyperbaton Proj/hyperTrans3OutputsMixtral7B.csv"  # Replace with the desired output CSV path
    hindi_column_name = "Hindi"  # Replace with the name of the column containing Hindi text
    spanish_column_name = "Spanish"  # Replace with the name of the column containing Spanish text
    french_column_name = "French"  # Replace with the name of the column containing French text

    process_csv_with_translations(input_csv_path, output_csv_path, hindi_column_name, spanish_column_name, french_column_name)

Streaming output truncated to the last 5000 lines.
Progress saved to /content/drive/MyDrive/GT/CS 8803 LLM/Hyperbaton Proj/hyperTrans3OutputsMixtral7B.csv
Processing row 85/2583
Progress saved to /content/drive/MyDrive/GT/CS 8803 LLM/Hyperbaton Proj/hyperTrans3OutputsMixtral7B.csv
Processing row 86/2583
Progress saved to /content/drive/MyDrive/GT/CS 8803 LLM/Hyperbaton Proj/hyperTrans3OutputsMixtral7B.csv
Processing row 87/2583
Progress saved to /content/drive/MyDrive/GT/CS 8803 LLM/Hyperbaton Proj/hyperTrans3OutputsMixtral7B.csv
Processing row 88/2583
Progress saved to /content/drive/MyDrive/GT/CS 8803 LLM/Hyperbaton Proj/hyperTrans3OutputsMixtral7B.csv
Processing row 89/2583
Progress saved to /content/drive/MyDrive/GT/CS 8803 LLM/Hyperbaton Proj/hyperTrans3OutputsMixtral7B.csv
Processing row 90/2583
Progress saved to /content/drive/MyDrive/GT/CS 8803 LLM/Hyperbaton Proj/hyperTrans3OutputsMixtral7B.csv
Processing row 91/2583
Progress saved to /content/drive/MyDrive/GT/CS 8803 LLM/Hype